In [112]:
using EzXML, DataFrames, LinearAlgebra, JuMP, Ipopt, Plots

In [3]:
function latLonEle(doc)
    gpx = elements(doc.root)
    trk = elements(gpx[2])
    trkseg = elements(trk[2])
    #allocate vectors for lat and long
    len = length(trkseg)
    lat = Vector{Float64}(undef,len)
    lon = similar(lat)
    ele = similar(lat)
    for i ∈ 1:len
        lat[i] = parse(Float64, trkseg[i]["lat"])
        lon[i] = parse(Float64, trkseg[i]["lon"])
        ele[i] = parse(Float64, elements(trkseg[i])[1].content)
    end
    return(lat, lon, ele)
end

latLonEle (generic function with 1 method)

In [42]:
function invHaversine(ϕ_1, λ_1, ϕ_2, λ_2)
    r = 6371000
    return(2*r*asin(sqrt(sin((ϕ_2-ϕ_1)/2)^2 + cos(ϕ_1)*cos(ϕ_2)*sin((λ_2-λ_1)/2)^2)))
end

invHaversine (generic function with 2 methods)

In [69]:
docUci = readxml("2021_uci_men.gpx")
docTok = readxml("2020_tokyo_men.gpx")
latUci, lonUci, eleUci = latLonEle(docUci)
latTok, lonTok, eleTok = latLonEle(docTok)
eleUci = eleUci .* 0.3048 #convert to meters, Tokyo is already in 
print(' ')

In [97]:
lenU = length(latUci)-1
lenT = length(latTok)-1
distUci = Vector{Float64}(undef,lenU)
eleChangeUci = similar(distUci)
distTok = Vector{Float64}(undef,lenT)
eleChangeTok = similar(distTok)
for i ∈ 1:lenU
    #find the distance between 2 waypoints using the inverse Haversine function
    distUci[i] = invHaversine(latUci[i], lonUci[i], latUci[i+1], lonUci[i+1])
    #find change in elevation between two waypoints
    eleChangeUci[i] = eleUci[i+1] - eleUci[i]
end
for i ∈ 1:lenT
    distTok[i] = invHaversine(latTok[i], lonTok[i], latTok[i+1], lonTok[i+1])
    if distTok[i] == 0.
        distTok[i] = 10.
    end
    eleChangeTok[i] = eleTok[i+1] - eleTok[i]
end

In [98]:
#We know total distances from the race materials so we will compute
#correction factors for both
distUci = (268300/sum(distUci)) .* distUci
distTok = (234000/sum(distTok)) .* distTok
#Compute grade in radians using inverse tangent of elevation/distance
gradeUci = atan.(eleChangeUci ./ distUci)
gradeTok = atan.(eleChangeTok ./ distTok)
print(' ')

In [114]:
#function for bisection step
#args are max power from power curve (initial guess),
#total energy(Pmax*tTot) from power curve, 
#xvec a vector of δx from one point to the next
#θvec a vector of grades from point to the next
function modelBuilder(Pmax, Etot, xvec, θvec)
    #set constants
    n = length(xvec)
    t_0 = 0. #s
    v_0 = 0. #m/s
    η = 0.95
    m = 70. #kg
    g = 9.8 #m/s^2
    A = 1.77 #m^2
    ρ = 1.225 #kg/m^3
    w = 2. #m/s
    E_end = 0.
    η_endur_init = 1.
    
    m = Model(Ipopt.Optimizer) #instantiate model with ipopt non-linear solver
    set_silent(m) #supress output
    
    @variables(m,begin
            #state variables
            v_i[1:n] ≥ 0. #m/s
            t_i[1:n] ≥ 0. #s
            E_i[1:n] ≥ 0. #J
            η_endur_i[1:n] ≥ 0.
            #control variable
            0. ≤ P_i[1:n] ≤ Pmax
        end)
    
    #goal is to minimize time
    @objective(m, Min, t_i[n])
    
    #Initial conditions, and the final condition that all energy must be gone
    fix(v_i[1], v_0; force = true)
    fix(t_i[1], t_0; force = true)
    fix(E_i[1], Etot; force = true)
    fix(E_i[n], E_end; force = true)
    fix(η_endur_i[1], η_endur_init; force = true)
    
    #functions which depend on state variables
    @NLexpressions(m,begin
            
        end)
    
    
end

#1 (generic function with 1 method)